In [22]:
import pandas as pd
import os
import nltk
# nltk.download('punkt')
nltk.download('stopwords')
# os.system('pip install contractions')

import contractions
import string
from nltk.corpus import stopwords, wordnet

from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/matt.turner/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
url = "https://raw.githubusercontent.com/mturner49/8600-deep-learning-project/main/data/CNN_Articels_clean.csv"

data = pd.read_csv(url)
data.head()

,Index,Author,Date published,Category,Section,Url,Headline,Description,Keywords,Second headline,Article text
0,0,"Jacopo Prisco, CNN",2021-07-15 02:46:59,news,world,https://www.cnn.com/2021/07/14/world/tusimple-...,"There's a shortage of truckers, but TuSimple t...",The e-commerce boom has exacerbated a global t...,"world, There's a shortage of truckers, but TuS...","There's a shortage of truckers, but TuSimple t...","(CNN)Right now, there's a shortage of truck d..."
1,2,"Stephanie Bailey, CNN",2021-05-12 07:52:09,news,world,https://www.cnn.com/2021/05/12/world/ironhand-...,Bioservo's robotic 'Ironhand' could protect fa...,Working in a factory can mean doing the same t...,"world, Bioservo's robotic 'Ironhand' could pro...",A robotic 'Ironhand' could protect factory wor...,(CNN)Working in a factory or warehouse can me...
2,3,"Words by Stephanie Bailey, video by Zahra Jamshed",2021-06-16 02:51:30,news,asia,https://www.cnn.com/2021/06/15/asia/swarm-robo...,This swarm of robots gets smarter the more it ...,"In a Hong Kong warehouse, a swarm of autonomou...","asia, This swarm of robots gets smarter the mo...",This swarm of robots gets smarter the more it ...,"(CNN)In a Hong Kong warehouse, a swarm of aut..."
3,4,"Paul R. La Monica, CNN Business",2022-03-15 09:57:36,business,investing,https://www.cnn.com/2022/03/15/investing/brics...,Russia is no longer an option for investors. T...,"For many years, the world's most popular emerg...","investing, Russia is no longer an option for i...",Russia is no longer an option for investors. T...,"New York (CNN Business)For many years, the wor..."
4,7,Reuters,2022-03-15 11:27:02,business,business,https://www.cnn.com/2022/03/15/business/russia...,Russian energy investment ban part of new EU s...,The European Union formally approved on Tuesda...,"business, Russian energy investment ban part o...",EU bans investment in Russian energy in new sa...,The European Union formally approved on Tuesda...


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4076 entries, 0 to 4075
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Index            4076 non-null   int64 
 1   Author           4076 non-null   object
 2   Date published   4076 non-null   object
 3   Category         4076 non-null   object
 4   Section          4076 non-null   object
 5   Url              4076 non-null   object
 6   Headline         4076 non-null   object
 7   Description      4076 non-null   object
 8   Keywords         4076 non-null   object
 9   Second headline  4076 non-null   object
 10  Article text     4076 non-null   object
dtypes: int64(1), object(10)
memory usage: 350.4+ KB


In [9]:
data['article_txt_no_contracts'] = data['Article text'].apply(lambda x: [contractions.fix(word) for word in x.split()])

data.head()

,Index,Author,Date published,Category,Section,Url,Headline,Description,Keywords,Second headline,Article text,article_txt_no_contracts
0,0,"Jacopo Prisco, CNN",2021-07-15 02:46:59,news,world,https://www.cnn.com/2021/07/14/world/tusimple-...,"There's a shortage of truckers, but TuSimple t...",The e-commerce boom has exacerbated a global t...,"world, There's a shortage of truckers, but TuS...","There's a shortage of truckers, but TuSimple t...","(CNN)Right now, there's a shortage of truck d...","[(CNN)Right, now,, there is, a, shortage, of, ..."
1,2,"Stephanie Bailey, CNN",2021-05-12 07:52:09,news,world,https://www.cnn.com/2021/05/12/world/ironhand-...,Bioservo's robotic 'Ironhand' could protect fa...,Working in a factory can mean doing the same t...,"world, Bioservo's robotic 'Ironhand' could pro...",A robotic 'Ironhand' could protect factory wor...,(CNN)Working in a factory or warehouse can me...,"[(CNN)Working, in, a, factory, or, warehouse, ..."
2,3,"Words by Stephanie Bailey, video by Zahra Jamshed",2021-06-16 02:51:30,news,asia,https://www.cnn.com/2021/06/15/asia/swarm-robo...,This swarm of robots gets smarter the more it ...,"In a Hong Kong warehouse, a swarm of autonomou...","asia, This swarm of robots gets smarter the mo...",This swarm of robots gets smarter the more it ...,"(CNN)In a Hong Kong warehouse, a swarm of aut...","[(CNN)In, a, Hong, Kong, warehouse,, a, swarm,..."
3,4,"Paul R. La Monica, CNN Business",2022-03-15 09:57:36,business,investing,https://www.cnn.com/2022/03/15/investing/brics...,Russia is no longer an option for investors. T...,"For many years, the world's most popular emerg...","investing, Russia is no longer an option for i...",Russia is no longer an option for investors. T...,"New York (CNN Business)For many years, the wor...","[New, York, (CNN, Business)For, many, years,, ..."
4,7,Reuters,2022-03-15 11:27:02,business,business,https://www.cnn.com/2022/03/15/business/russia...,Russian energy investment ban part of new EU s...,The European Union formally approved on Tuesda...,"business, Russian energy investment ban part o...",EU bans investment in Russian energy in new sa...,The European Union formally approved on Tuesda...,"[The, European, Union, formally, approved, on,..."


In [10]:
data['article_txt_str'] = [' '.join(map(str, l)) for l in data['article_txt_no_contracts']]
data.head()

,Index,Author,Date published,Category,Section,Url,Headline,Description,Keywords,Second headline,Article text,article_txt_no_contracts,article_txt_str
0,0,"Jacopo Prisco, CNN",2021-07-15 02:46:59,news,world,https://www.cnn.com/2021/07/14/world/tusimple-...,"There's a shortage of truckers, but TuSimple t...",The e-commerce boom has exacerbated a global t...,"world, There's a shortage of truckers, but TuS...","There's a shortage of truckers, but TuSimple t...","(CNN)Right now, there's a shortage of truck d...","[(CNN)Right, now,, there is, a, shortage, of, ...","(CNN)Right now, there is a shortage of truck d..."
1,2,"Stephanie Bailey, CNN",2021-05-12 07:52:09,news,world,https://www.cnn.com/2021/05/12/world/ironhand-...,Bioservo's robotic 'Ironhand' could protect fa...,Working in a factory can mean doing the same t...,"world, Bioservo's robotic 'Ironhand' could pro...",A robotic 'Ironhand' could protect factory wor...,(CNN)Working in a factory or warehouse can me...,"[(CNN)Working, in, a, factory, or, warehouse, ...",(CNN)Working in a factory or warehouse can mea...
2,3,"Words by Stephanie Bailey, video by Zahra Jamshed",2021-06-16 02:51:30,news,asia,https://www.cnn.com/2021/06/15/asia/swarm-robo...,This swarm of robots gets smarter the more it ...,"In a Hong Kong warehouse, a swarm of autonomou...","asia, This swarm of robots gets smarter the mo...",This swarm of robots gets smarter the more it ...,"(CNN)In a Hong Kong warehouse, a swarm of aut...","[(CNN)In, a, Hong, Kong, warehouse,, a, swarm,...","(CNN)In a Hong Kong warehouse, a swarm of auto..."
3,4,"Paul R. La Monica, CNN Business",2022-03-15 09:57:36,business,investing,https://www.cnn.com/2022/03/15/investing/brics...,Russia is no longer an option for investors. T...,"For many years, the world's most popular emerg...","investing, Russia is no longer an option for i...",Russia is no longer an option for investors. T...,"New York (CNN Business)For many years, the wor...","[New, York, (CNN, Business)For, many, years,, ...","New York (CNN Business)For many years, the wor..."
4,7,Reuters,2022-03-15 11:27:02,business,business,https://www.cnn.com/2022/03/15/business/russia...,Russian energy investment ban part of new EU s...,The European Union formally approved on Tuesda...,"business, Russian energy investment ban part o...",EU bans investment in Russian energy in new sa...,The European Union formally approved on Tuesda...,"[The, European, Union, formally, approved, on,...",The European Union formally approved on Tuesda...


In [14]:
data['tokenized_article_txt'] = data['article_txt_str'].apply(word_tokenize)
data.head()

,Index,Author,Date published,Category,Section,Url,Headline,Description,Keywords,Second headline,Article text,article_txt_no_contracts,article_txt_str,tokenized_article_txt
0,0,"Jacopo Prisco, CNN",2021-07-15 02:46:59,news,world,https://www.cnn.com/2021/07/14/world/tusimple-...,"There's a shortage of truckers, but TuSimple t...",The e-commerce boom has exacerbated a global t...,"world, There's a shortage of truckers, but TuS...","There's a shortage of truckers, but TuSimple t...","(CNN)Right now, there's a shortage of truck d...","[(CNN)Right, now,, there is, a, shortage, of, ...","(CNN)Right now, there is a shortage of truck d...","[(, CNN, ), Right, now, ,, there, is, a, short..."
1,2,"Stephanie Bailey, CNN",2021-05-12 07:52:09,news,world,https://www.cnn.com/2021/05/12/world/ironhand-...,Bioservo's robotic 'Ironhand' could protect fa...,Working in a factory can mean doing the same t...,"world, Bioservo's robotic 'Ironhand' could pro...",A robotic 'Ironhand' could protect factory wor...,(CNN)Working in a factory or warehouse can me...,"[(CNN)Working, in, a, factory, or, warehouse, ...",(CNN)Working in a factory or warehouse can mea...,"[(, CNN, ), Working, in, a, factory, or, wareh..."
2,3,"Words by Stephanie Bailey, video by Zahra Jamshed",2021-06-16 02:51:30,news,asia,https://www.cnn.com/2021/06/15/asia/swarm-robo...,This swarm of robots gets smarter the more it ...,"In a Hong Kong warehouse, a swarm of autonomou...","asia, This swarm of robots gets smarter the mo...",This swarm of robots gets smarter the more it ...,"(CNN)In a Hong Kong warehouse, a swarm of aut...","[(CNN)In, a, Hong, Kong, warehouse,, a, swarm,...","(CNN)In a Hong Kong warehouse, a swarm of auto...","[(, CNN, ), In, a, Hong, Kong, warehouse, ,, a..."
3,4,"Paul R. La Monica, CNN Business",2022-03-15 09:57:36,business,investing,https://www.cnn.com/2022/03/15/investing/brics...,Russia is no longer an option for investors. T...,"For many years, the world's most popular emerg...","investing, Russia is no longer an option for i...",Russia is no longer an option for investors. T...,"New York (CNN Business)For many years, the wor...","[New, York, (CNN, Business)For, many, years,, ...","New York (CNN Business)For many years, the wor...","[New, York, (, CNN, Business, ), For, many, ye..."
4,7,Reuters,2022-03-15 11:27:02,business,business,https://www.cnn.com/2022/03/15/business/russia...,Russian energy investment ban part of new EU s...,The European Union formally approved on Tuesda...,"business, Russian energy investment ban part o...",EU bans investment in Russian energy in new sa...,The European Union formally approved on Tuesda...,"[The, European, Union, formally, approved, on,...",The European Union formally approved on Tuesda...,"[The, European, Union, formally, approved, on,..."


In [15]:
data['lower'] = data['tokenized_article_txt'].apply(lambda x: [word.lower() for word in x])
data.head()

,Index,Author,Date published,Category,Section,Url,Headline,Description,Keywords,Second headline,Article text,article_txt_no_contracts,article_txt_str,tokenized_article_txt,lower
0,0,"Jacopo Prisco, CNN",2021-07-15 02:46:59,news,world,https://www.cnn.com/2021/07/14/world/tusimple-...,"There's a shortage of truckers, but TuSimple t...",The e-commerce boom has exacerbated a global t...,"world, There's a shortage of truckers, but TuS...","There's a shortage of truckers, but TuSimple t...","(CNN)Right now, there's a shortage of truck d...","[(CNN)Right, now,, there is, a, shortage, of, ...","(CNN)Right now, there is a shortage of truck d...","[(, CNN, ), Right, now, ,, there, is, a, short...","[(, cnn, ), right, now, ,, there, is, a, short..."
1,2,"Stephanie Bailey, CNN",2021-05-12 07:52:09,news,world,https://www.cnn.com/2021/05/12/world/ironhand-...,Bioservo's robotic 'Ironhand' could protect fa...,Working in a factory can mean doing the same t...,"world, Bioservo's robotic 'Ironhand' could pro...",A robotic 'Ironhand' could protect factory wor...,(CNN)Working in a factory or warehouse can me...,"[(CNN)Working, in, a, factory, or, warehouse, ...",(CNN)Working in a factory or warehouse can mea...,"[(, CNN, ), Working, in, a, factory, or, wareh...","[(, cnn, ), working, in, a, factory, or, wareh..."
2,3,"Words by Stephanie Bailey, video by Zahra Jamshed",2021-06-16 02:51:30,news,asia,https://www.cnn.com/2021/06/15/asia/swarm-robo...,This swarm of robots gets smarter the more it ...,"In a Hong Kong warehouse, a swarm of autonomou...","asia, This swarm of robots gets smarter the mo...",This swarm of robots gets smarter the more it ...,"(CNN)In a Hong Kong warehouse, a swarm of aut...","[(CNN)In, a, Hong, Kong, warehouse,, a, swarm,...","(CNN)In a Hong Kong warehouse, a swarm of auto...","[(, CNN, ), In, a, Hong, Kong, warehouse, ,, a...","[(, cnn, ), in, a, hong, kong, warehouse, ,, a..."
3,4,"Paul R. La Monica, CNN Business",2022-03-15 09:57:36,business,investing,https://www.cnn.com/2022/03/15/investing/brics...,Russia is no longer an option for investors. T...,"For many years, the world's most popular emerg...","investing, Russia is no longer an option for i...",Russia is no longer an option for investors. T...,"New York (CNN Business)For many years, the wor...","[New, York, (CNN, Business)For, many, years,, ...","New York (CNN Business)For many years, the wor...","[New, York, (, CNN, Business, ), For, many, ye...","[new, york, (, cnn, business, ), for, many, ye..."
4,7,Reuters,2022-03-15 11:27:02,business,business,https://www.cnn.com/2022/03/15/business/russia...,Russian energy investment ban part of new EU s...,The European Union formally approved on Tuesda...,"business, Russian energy investment ban part o...",EU bans investment in Russian energy in new sa...,The European Union formally approved on Tuesda...,"[The, European, Union, formally, approved, on,...",The European Union formally approved on Tuesda...,"[The, European, Union, formally, approved, on,...","[the, european, union, formally, approved, on,..."


In [19]:
punc = string.punctuation
data['no_punc'] = data['lower'].apply(lambda x: [word for word in x if word not in punc])
data.head()

,Index,Author,Date published,Category,Section,Url,Headline,Description,Keywords,Second headline,Article text,article_txt_no_contracts,article_txt_str,tokenized_article_txt,lower,no_punc
0,0,"Jacopo Prisco, CNN",2021-07-15 02:46:59,news,world,https://www.cnn.com/2021/07/14/world/tusimple-...,"There's a shortage of truckers, but TuSimple t...",The e-commerce boom has exacerbated a global t...,"world, There's a shortage of truckers, but TuS...","There's a shortage of truckers, but TuSimple t...","(CNN)Right now, there's a shortage of truck d...","[(CNN)Right, now,, there is, a, shortage, of, ...","(CNN)Right now, there is a shortage of truck d...","[(, CNN, ), Right, now, ,, there, is, a, short...","[(, cnn, ), right, now, ,, there, is, a, short...","[cnn, right, now, there, is, a, shortage, of, ..."
1,2,"Stephanie Bailey, CNN",2021-05-12 07:52:09,news,world,https://www.cnn.com/2021/05/12/world/ironhand-...,Bioservo's robotic 'Ironhand' could protect fa...,Working in a factory can mean doing the same t...,"world, Bioservo's robotic 'Ironhand' could pro...",A robotic 'Ironhand' could protect factory wor...,(CNN)Working in a factory or warehouse can me...,"[(CNN)Working, in, a, factory, or, warehouse, ...",(CNN)Working in a factory or warehouse can mea...,"[(, CNN, ), Working, in, a, factory, or, wareh...","[(, cnn, ), working, in, a, factory, or, wareh...","[cnn, working, in, a, factory, or, warehouse, ..."
2,3,"Words by Stephanie Bailey, video by Zahra Jamshed",2021-06-16 02:51:30,news,asia,https://www.cnn.com/2021/06/15/asia/swarm-robo...,This swarm of robots gets smarter the more it ...,"In a Hong Kong warehouse, a swarm of autonomou...","asia, This swarm of robots gets smarter the mo...",This swarm of robots gets smarter the more it ...,"(CNN)In a Hong Kong warehouse, a swarm of aut...","[(CNN)In, a, Hong, Kong, warehouse,, a, swarm,...","(CNN)In a Hong Kong warehouse, a swarm of auto...","[(, CNN, ), In, a, Hong, Kong, warehouse, ,, a...","[(, cnn, ), in, a, hong, kong, warehouse, ,, a...","[cnn, in, a, hong, kong, warehouse, a, swarm, ..."
3,4,"Paul R. La Monica, CNN Business",2022-03-15 09:57:36,business,investing,https://www.cnn.com/2022/03/15/investing/brics...,Russia is no longer an option for investors. T...,"For many years, the world's most popular emerg...","investing, Russia is no longer an option for i...",Russia is no longer an option for investors. T...,"New York (CNN Business)For many years, the wor...","[New, York, (CNN, Business)For, many, years,, ...","New York (CNN Business)For many years, the wor...","[New, York, (, CNN, Business, ), For, many, ye...","[new, york, (, cnn, business, ), for, many, ye...","[new, york, cnn, business, for, many, years, t..."
4,7,Reuters,2022-03-15 11:27:02,business,business,https://www.cnn.com/2022/03/15/business/russia...,Russian energy investment ban part of new EU s...,The European Union formally approved on Tuesda...,"business, Russian energy investment ban part o...",EU bans investment in Russian energy in new sa...,The European Union formally approved on Tuesda...,"[The, European, Union, formally, approved, on,...",The European Union formally approved on Tuesda...,"[The, European, Union, formally, approved, on,...","[the, european, union, formally, approved, on,...","[the, european, union, formally, approved, on,..."


In [23]:
stop_words = set(stopwords.words('english'))
data['stopwords_removed'] = data['no_punc'].apply(lambda x: [word for word in x if word not in stop_words])
data.head()

,Index,Author,Date published,Category,Section,Url,Headline,Description,Keywords,Second headline,Article text,article_txt_no_contracts,article_txt_str,tokenized_article_txt,lower,no_punc,stopwords_removed
0,0,"Jacopo Prisco, CNN",2021-07-15 02:46:59,news,world,https://www.cnn.com/2021/07/14/world/tusimple-...,"There's a shortage of truckers, but TuSimple t...",The e-commerce boom has exacerbated a global t...,"world, There's a shortage of truckers, but TuS...","There's a shortage of truckers, but TuSimple t...","(CNN)Right now, there's a shortage of truck d...","[(CNN)Right, now,, there is, a, shortage, of, ...","(CNN)Right now, there is a shortage of truck d...","[(, CNN, ), Right, now, ,, there, is, a, short...","[(, cnn, ), right, now, ,, there, is, a, short...","[cnn, right, now, there, is, a, shortage, of, ...","[cnn, right, shortage, truck, drivers, us, wor..."
1,2,"Stephanie Bailey, CNN",2021-05-12 07:52:09,news,world,https://www.cnn.com/2021/05/12/world/ironhand-...,Bioservo's robotic 'Ironhand' could protect fa...,Working in a factory can mean doing the same t...,"world, Bioservo's robotic 'Ironhand' could pro...",A robotic 'Ironhand' could protect factory wor...,(CNN)Working in a factory or warehouse can me...,"[(CNN)Working, in, a, factory, or, warehouse, ...",(CNN)Working in a factory or warehouse can mea...,"[(, CNN, ), Working, in, a, factory, or, wareh...","[(, cnn, ), working, in, a, factory, or, wareh...","[cnn, working, in, a, factory, or, warehouse, ...","[cnn, working, factory, warehouse, mean, task,..."
2,3,"Words by Stephanie Bailey, video by Zahra Jamshed",2021-06-16 02:51:30,news,asia,https://www.cnn.com/2021/06/15/asia/swarm-robo...,This swarm of robots gets smarter the more it ...,"In a Hong Kong warehouse, a swarm of autonomou...","asia, This swarm of robots gets smarter the mo...",This swarm of robots gets smarter the more it ...,"(CNN)In a Hong Kong warehouse, a swarm of aut...","[(CNN)In, a, Hong, Kong, warehouse,, a, swarm,...","(CNN)In a Hong Kong warehouse, a swarm of auto...","[(, CNN, ), In, a, Hong, Kong, warehouse, ,, a...","[(, cnn, ), in, a, hong, kong, warehouse, ,, a...","[cnn, in, a, hong, kong, warehouse, a, swarm, ...","[cnn, hong, kong, warehouse, swarm, autonomous..."
3,4,"Paul R. La Monica, CNN Business",2022-03-15 09:57:36,business,investing,https://www.cnn.com/2022/03/15/investing/brics...,Russia is no longer an option for investors. T...,"For many years, the world's most popular emerg...","investing, Russia is no longer an option for i...",Russia is no longer an option for investors. T...,"New York (CNN Business)For many years, the wor...","[New, York, (CNN, Business)For, many, years,, ...","New York (CNN Business)For many years, the wor...","[New, York, (, CNN, Business, ), For, many, ye...","[new, york, (, cnn, business, ), for, many, ye...","[new, york, cnn, business, for, many, years, t...","[new, york, cnn, business, many, years, world,..."
4,7,Reuters,2022-03-15 11:27:02,business,business,https://www.cnn.com/2022/03/15/business/russia...,Russian energy investment ban part of new EU s...,The European Union formally approved on Tuesda...,"business, Russian energy investment ban part o...",EU bans investment in Russian energy in new sa...,The European Union formally approved on Tuesda...,"[The, European, Union, formally, approved, on,...",The European Union formally approved on Tuesda...,"[The, European, Union, formally, approved, on,...","[the, european, union, formally, approved, on,...","[the, european, union, formally, approved, on,...","[european, union, formally, approved, tuesday,..."
